## Exploring different Document Processing Models
 
 **Goal**: An Agentic approach to explore different Document Processing Models for OCR, Layout Analysis, and Table Extraction.
 
 **Output**: A simple langgraph agent that explores different ocr techniques/models.


In [ ]:
# Setting up a react agent with LangChain


from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import AIMessageChunk, HumanMessage
from langchain_core.runnables import RunnableConfig

# from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import InMemorySaver
from langsmith import traceable
from prompts import DOCUMENT_AGENT_SYSTEM_PROMPT, USER_MESSAGE_TEMPLATE

load_dotenv()
# conn = connect("langgraph_checkpoints.db")
# checkpointer = SqliteSaver(conn=conn)
checkpointer = InMemorySaver()
llm = init_chat_model(
    "anthropic:claude-sonnet-4-5-20250929", temperature=1, thinking={"type": "enabled", "budget_tokens": 10000}
)

document_agent = create_agent(
    model=llm,
    system_prompt=DOCUMENT_AGENT_SYSTEM_PROMPT,
    checkpointer=checkpointer,
)
config = RunnableConfig(configurable={"thread_id": "test_thread"})
inputs = {
    "messages": [
        HumanMessage(USER_MESSAGE_TEMPLATE.format(message="Hello, world!", document_path="No Document Uploaded"))
    ]
}


@traceable(name="DocumentAgentInteraction")
def agent_execution():
    """Execute the document agent and streams the response."""
    ai_message = AIMessageChunk(content=[])

    for _, chunk in document_agent.stream(inputs, stream_mode=["messages"], config=config):
        ai_message += chunk[0]

    ai_message.pretty_print()
    return ai_message


agent_execution()

============================ Aimessagechunk Message ============================

[{'thinking': 'The user has greeted with "Hello, world!" and the system message indicates that no document has been uploaded yet. The user is being told to process a document using all available OCR techniques, but there\'s no document path provided (it says "No Document Uploaded").\n\nI should:\n1. Greet the user back\n2. Acknowledge that no document has been uploaded yet\n3. Explain what I can do once they upload a document\n4. Let them know about the available OCR tools I can use\n\nI need to be helpful and clear about what\'s needed next.', 'type': 'thinking', 'index': 0, 'signature': 'Eq8FCkYICxgCKkBITo/AMKVWu8GCLRLl3Twm6h8wMXhJBrKPGLzNxS0iOdVd/SJ1hSrM03zmDjArJUS0uTY+Km7BX/G4r4fhoO9hEgx6E3JOvOza9yid7T4aDNcuo4udA6zYTFDtxyIwVjD3LxfpDGCxEStO09Cqbao2SmLvV2hQlwhzXKsxZEtLAYzMCwBNlX8dF4SDGuQqKpYEBAYvr0L1HM9yQSD/her+dNsiQGJCFiIiPYbpmVr2IQ7tOzND3IbpcCl4vXTnbV1YX3CT8qN9cii1aO1NIzJ8i3SeRaApw1ruwMb1vuyH+LbPkVbJI

AIMessageChunk(content=[{'thinking': 'The user has greeted with "Hello, world!" and the system message indicates that no document has been uploaded yet. The user is being told to process a document using all available OCR techniques, but there\'s no document path provided (it says "No Document Uploaded").\n\nI should:\n1. Greet the user back\n2. Acknowledge that no document has been uploaded yet\n3. Explain what I can do once they upload a document\n4. Let them know about the available OCR tools I can use\n\nI need to be helpful and clear about what\'s needed next.', 'type': 'thinking', 'index': 0, 'signature': 'Eq8FCkYICxgCKkBITo/AMKVWu8GCLRLl3Twm6h8wMXhJBrKPGLzNxS0iOdVd/SJ1hSrM03zmDjArJUS0uTY+Km7BX/G4r4fhoO9hEgx6E3JOvOza9yid7T4aDNcuo4udA6zYTFDtxyIwVjD3LxfpDGCxEStO09Cqbao2SmLvV2hQlwhzXKsxZEtLAYzMCwBNlX8dF4SDGuQqKpYEBAYvr0L1HM9yQSD/her+dNsiQGJCFiIiPYbpmVr2IQ7tOzND3IbpcCl4vXTnbV1YX3CT8qN9cii1aO1NIzJ8i3SeRaApw1ruwMb1vuyH+LbPkVbJI6WlwjcUcOhsaZ34hpKVJAAVbWMnWlXjU0Qo/b6ENmVF/QG2XungQ6PZrcmT